In [4]:
import h5py
import hdf5plugin
import blimpy
from blimpy import Waterfall
from blimpy import utils
from blimpy import plotting
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import glob
import turbo_seti.find_doppler.seti_event as turbo
import turbo_seti.find_event as find_event
from turbo_seti.find_doppler.find_doppler import FindDoppler
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline
import pandas as pd
%matplotlib inline

print("All packages imported!")

All packages imported!


In [6]:
all_events = pd.read_csv('/home/cgchoza/BL-TESSsearch/analysis/event_data.csv')
pd.set_option('display.max_columns', None)


In [7]:
all_files = np.array(all_events["FileID"])
len(all_files)

5519

In [8]:
all_file_paths = pd.read_csv('/home/cgchoza/complete_cadences_catalog.csv')
all_h5_paths = np.array(all_file_paths['.h5 path'])
len(all_h5_paths)

36558

In [9]:
event_file_paths = [i for i in all_h5_paths if i.split("/")[5] in all_files]

In [16]:
all_files[2000]

'spliced_blc10111213141516o7o0212223242526o7o031323334353637_guppi_58868_52090_NGC5831_0056.gpuspec.0000.dat'

In [14]:
len([i for i in all_events['FileID'] if 'h5' in i])

1101

In [58]:
all_events

,Unnamed: 0.1,Unnamed: 0,TopHitNum,DriftRate,SNR,Freq,ChanIndx,FreqStart,FreqEnd,CoarseChanNum,FullNumHitsInRange,FileID,Source,MJD,RA,DEC,DELTAT,DELTAF,max_drift_rate,obs_length,Hit_ID,status,in_n_ons,RFI_in_range,delta_t,Band,FilePath
0,0,38,39,-0.004938,13.087125,2223.152282,696276,2223.152282,2223.152282,9,1,spliced_blc0001020304050607_guppi_58306_31766_...,DDO210,58306.367662,20.78102,-12.8485,18.253611,-2.793968,4.0,584.115552,DDO210_38,on_table_1,2.0,0,0.0,L,/home/obs/events/24777/2251/found_event_table.csv
1,1,2176,41,-0.004938,13.687256,2223.152277,696278,2223.152277,2223.152277,9,1,spliced_blc0001020304050607_guppi_58306_33010_...,DDO210,58306.382060,20.781207000000002,-12.8477,18.253611,-2.793968,4.0,584.115552,DDO210_38,on_table_2,NaN,0,1244.0,L,/home/obs/events/24777/2251/found_event_table.csv
2,2,4258,32,-0.004938,32.954437,2223.152271,696280,2223.152271,2223.152271,9,1,spliced_blc0001020304050607_guppi_58306_34254_...,DDO210,58306.396458,20.781026999999998,-12.8477,18.253611,-2.793968,4.0,584.115552,DDO210_38,on_table_3,NaN,0,2488.0,L,/home/obs/events/24777/2251/found_event_table.csv
3,3,459,460,-2.715649,28.688183,1618.354897,106460,1618.354897,1618.354897,216,1,spliced_blc0001020304050607_guppi_58306_31766_...,DDO210,58306.367662,20.78102,-12.8485,18.253611,-2.793968,4.0,584.115552,DDO210_459,on_table_1,2.0,0,0.0,L,/home/obs/events/24777/2251/found_event_table.csv
4,4,2578,443,0.967758,11.333396,1618.352712,107242,1618.352712,1618.352712,216,1,spliced_blc0001020304050607_guppi_58306_33010_...,DDO210,58306.382060,20.781207000000002,-12.8477,18.253611,-2.793968,4.0,584.115552,DDO210_459,on_table_2,NaN,0,1244.0,L,/home/obs/events/24777/2251/found_event_table.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,16,7017,3910,0.009566,44.257126,10967.600693,935641,10967.602327,10967.599064,32,50249,blc01_guppi_59414_22460_WLM_0103.rawspec.0000.h5,WLM,59414.259954,0h01m55.344s,-15d26m43.08s,18.253611,-2.793968,4.0,292.057776,WLM_2628,on_table_2,NaN,0,628.0,X,/datax/scratch/cgchoza/event_files/395338_blc0...
5515,17,15978,8327,0.009566,153.227982,10967.600699,935639,10967.602333,10967.599070,32,2209020,blc01_guppi_59414_23104_WLM_0105.rawspec.0000.h5,WLM,59414.267407,0h01m58.968s,-15d27m05.76s,18.253611,-2.793968,4.0,292.057776,WLM_2628,on_table_3,NaN,0,1272.0,X,/datax/scratch/cgchoza/event_files/395338_blc0...
5516,18,3067,3068,0.009566,15.694656,10959.777293,590017,10959.778927,10959.775664,35,512,blc01_guppi_59414_21832_WLM_0101.rawspec.0000.h5,WLM,59414.252685,0h01m58.104s,-15d27m38.88s,18.253611,-2.793968,4.0,292.057776,WLM_3067,on_table_1,2.0,0,0.0,X,/datax/scratch/cgchoza/event_files/395338_blc0...
5517,19,7584,4477,0.009566,29.186354,10959.777301,590014,10959.778936,10959.775672,35,15247,blc01_guppi_59414_22460_WLM_0103.rawspec.0000.h5,WLM,59414.259954,0h01m55.344s,-15d26m43.08s,18.253611,-2.793968,4.0,292.057776,WLM_3067,on_table_2,NaN,0,628.0,X,/datax/scratch/cgchoza/event_files/395338_blc0...
